In [3]:
import tensorflow as tf
import numpy as np
import keras
import pandas as pd
import os

In [2]:
!unzip -q QPSK.zip

In [6]:
x = []
y = []
for i in os.listdir("/content/QPSK/bch"):
  print("Compiling file :", i)
  conv_0 = pd.read_csv("/content/QPSK/bch/"+i, header=None)
  for j in range(100):
    x.append(conv_0.iloc[:200, j].values)
    y.append(0)

for i in os.listdir("/content/QPSK/conv"):
  print("Compiling file :", i)
  conv_0 = pd.read_csv("/content/QPSK/conv/"+i, header=None)
  for j in range(100):
    x.append(conv_0.iloc[:200, j].values)
    y.append(1)

for i in os.listdir("/content/QPSK/tpc"):
  print("Compiling file :", i)
  conv_0 = pd.read_csv("/content/QPSK/tpc/"+i, header=None)
  for j in range(100):
    x.append(conv_0.iloc[:200, j].values)
    y.append(2)

for i in os.listdir("/content/QPSK/turbo"):
  print("Compiling file :", i)
  conv_0 = pd.read_csv("/content/QPSK/turbo/"+i, header=None)
  for j in range(50):
    x.append(conv_0.iloc[:200, j].values)
    y.append(3)

X = np.array(x)
X.shape

Compiling file : bch_2snr.csv
Compiling file : bch_10snr.csv
Compiling file : bch_3snr.csv
Compiling file : bch_9snr.csv
Compiling file : bch_7snr.csv
Compiling file : bch_8snr.csv
Compiling file : bch_6snr.csv
Compiling file : bch_1snr.csv
Compiling file : bch_4snr.csv
Compiling file : bch_0snr.csv
Compiling file : bch_5snr.csv
Compiling file : conv_5snr.csv
Compiling file : conv_4snr.csv
Compiling file : conv_6snr.csv
Compiling file : conv_9snr.csv
Compiling file : conv_0snr.csv
Compiling file : conv_3snr.csv
Compiling file : conv_1snr.csv
Compiling file : conv_10snr.csv
Compiling file : conv_2snr.csv
Compiling file : conv_8snr.csv
Compiling file : conv_7snr.csv
Compiling file : tpc_5snr.csv
Compiling file : tpc_3snr.csv
Compiling file : tpc_8snr.csv
Compiling file : tpc_10snr.csv
Compiling file : tpc_9snr.csv
Compiling file : tpc_6snr.csv
Compiling file : tpc_0snr.csv
Compiling file : tpc_2snr.csv
Compiling file : tpc_1snr.csv
Compiling file : tpc_7snr.csv
Compiling file : tpc_4snr.

(3850, 200)

In [7]:
x_data = []
for i in x:
  x_data.append([[x] for x in i])

x_data = np.array(x_data)

In [8]:
x_data.shape

(3850, 200, 1)

In [13]:
Y = np.array(y)

In [17]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_data, Y, test_size=0.25, shuffle=True, stratify=Y)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((2887, 200, 1), (963, 200, 1), (2887,), (963,))

In [18]:
x_t, x_v, y_t, y_v = train_test_split(x_test, y_test, test_size=0.50, shuffle=True, stratify=y_test)
x_t.shape, x_v.shape, y_t.shape, y_v.shape

((481, 200, 1), (482, 200, 1), (481,), (482,))

In [20]:
inp_layer = keras.layers.Input(shape=(200,1))

#conv1-64, 3x1
x = keras.layers.Conv1D(filters=64, kernel_size=3)(inp_layer)
x_act = keras.layers.Activation(tf.nn.relu)(x)

#conv1-64, 3x1
x = keras.layers.Conv1D(filters=64, kernel_size=3)(x_act)
x_act = keras.layers.Activation(tf.nn.relu)(x)

#conv1-128, 3x1
x = keras.layers.Conv1D(filters=128, kernel_size=3)(x_act)
x_act = keras.layers.Activation(tf.nn.relu)(x)

#global average pooling
gap = keras.layers.GlobalAveragePooling1D()(x_act)

#dense
d1 = keras.layers.Dense(64, activation="relu")(gap)
d2 = keras.layers.Dense(64, activation="relu")(d1)
out_layer = keras.layers.Dense(4, activation="sigmoid")(d2)

model_test1 = keras.Model(inp_layer, out_layer)

model_test1.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 200, 1)]          0         
                                                                 
 conv1d_3 (Conv1D)           (None, 198, 64)           256       
                                                                 
 activation_3 (Activation)   (None, 198, 64)           0         
                                                                 
 conv1d_4 (Conv1D)           (None, 196, 64)           12352     
                                                                 
 activation_4 (Activation)   (None, 196, 64)           0         
                                                                 
 conv1d_5 (Conv1D)           (None, 194, 128)          24704     
                                                                 
 activation_5 (Activation)   (None, 194, 128)          0   

In [21]:
model_test1.compile(
    optimizer="adam",
    loss = "sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [22]:
callbacks=[
    tf.keras.callbacks.ModelCheckpoint("model_qpsk"),
    tf.keras.callbacks.TensorBoard("logs_qpsk")
]

In [24]:
model_test1.fit(x_train, y_train, epochs=50, validation_data=(x_v, y_v), callbacks=callbacks)

Epoch 1/50
91/91 [==============================] - 5s 57ms/step - loss: 0.2706 - accuracy: 0.8795 - val_loss: 0.3340 - val_accuracy: 0.8734
Epoch 2/50
91/91 [==============================] - 5s 57ms/step - loss: 0.2739 - accuracy: 0.8788 - val_loss: 0.3568 - val_accuracy: 0.8589
Epoch 3/50
91/91 [==============================] - 7s 82ms/step - loss: 0.2870 - accuracy: 0.8649 - val_loss: 0.3326 - val_accuracy: 0.8672
Epoch 4/50
91/91 [==============================] - 5s 52ms/step - loss: 0.2646 - accuracy: 0.8899 - val_loss: 0.3224 - val_accuracy: 0.8672
Epoch 5/50
91/91 [==============================] - 5s 59ms/step - loss: 0.2635 - accuracy: 0.8822 - val_loss: 0.3126 - val_accuracy: 0.8859
Epoch 6/50
91/91 [==============================] - 5s 59ms/step - loss: 0.2590 - accuracy: 0.8847 - val_loss: 0.3155 - val_accuracy: 0.8838
Epoch 7/50
91/91 [==============================] - 5s 57ms/step - loss: 0.2670 - accuracy: 0.8826 - val_loss: 0.3467 - val_accuracy: 0.8776
Epoch 8/50
91

In [25]:
model_test1.evaluate(x_t, y_t)

16/16 [==============================] - 0s 21ms/step - loss: 0.2621 - accuracy: 0.8815


[0.2621367275714874, 0.8814969062805176]

In [26]:
model_test1.save("model_qpsk_92acc")

In [28]:
!zip -r /content/model_qpsk92acc.zip /content/model_qpsk_92acc

  adding: content/model_qpsk_92acc/ (stored 0%)
  adding: content/model_qpsk_92acc/variables/ (stored 0%)
  adding: content/model_qpsk_92acc/variables/variables.index (deflated 65%)
  adding: content/model_qpsk_92acc/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: content/model_qpsk_92acc/keras_metadata.pb (deflated 91%)
  adding: content/model_qpsk_92acc/saved_model.pb (deflated 88%)
  adding: content/model_qpsk_92acc/assets/ (stored 0%)
  adding: content/model_qpsk_92acc/fingerprint.pb (stored 0%)


In [29]:
!zip -r /content/logs_qpsk92acc.zip /content/logs_qpsk

  adding: content/logs_qpsk/ (stored 0%)
  adding: content/logs_qpsk/validation/ (stored 0%)
  adding: content/logs_qpsk/validation/events.out.tfevents.1703048084.d8a75a6848b3.281.3.v2 (deflated 78%)
  adding: content/logs_qpsk/validation/events.out.tfevents.1703047511.d8a75a6848b3.281.1.v2 (deflated 79%)
  adding: content/logs_qpsk/train/ (stored 0%)
  adding: content/logs_qpsk/train/events.out.tfevents.1703047504.d8a75a6848b3.281.0.v2 (deflated 86%)
  adding: content/logs_qpsk/train/events.out.tfevents.1703048081.d8a75a6848b3.281.2.v2 (deflated 86%)
